## Midterm BME 502 2025

In this midterm exam, you are tasked to develop a method to tell whether data is better described by a line fit or a sigmoidal fit.

In [ ]:
import Pkg
Pkg.activate(".")

In [ ]:
Pkg.add(["Plots","Distributions","Random","Statistics","NonlinearSolve","ForwardDiff", "Optim"])

In [ ]:
using Plots, Distributions, Random, Statistics, NonlinearSolve, ForwardDiff, LinearAlgebra

In [ ]:
line_fit(x,p) = p[1] .+ p[2] .* x

In [ ]:
sigmoid_fit(x,p) = p[1] .+ p[2] ./ (1 .+ exp.(-p[3]*(x .- p[4])))

Let's create some plots to see how these functions look

In [ ]:
x = 0:0.1:10

In [ ]:
y_line = line_fit(x,[2,0.5])

In [ ]:
y_sigmoid = sigmoid_fit(x,[2.5,3.5,0.1,5.5])

In [ ]:
y_line = line_fit(x,[2,0.5])
y_sigmoid = sigmoid_fit(x,[2.2,4.7,1,5.0])
plot(x, y_line, label = "line")
plot!(x, y_sigmoid, label = "sigmoid")

Let us add some serious noise to it

In [ ]:
nd = Normal(0,5)
x_line = 10*rand(200)
y_line_n = line_fit(x_line,[2,0.5]) .+ rand(nd,length(x_line))
x_sigmoid = 10*rand(200)
y_sigmoid_n = sigmoid_fit(x_sigmoid,[2.2,4.7,1,5.0]) .+ rand(nd,length(x_sigmoid))

In [ ]:
plot(x, y_line, label = "line")
scatter!(x_line, y_line_n, label = "line data")

In [ ]:
plot(x, y_sigmoid, label = "sigmoid")
scatter!(x_sigmoid, y_sigmoid_n, label = "data")

In [ ]:
# let's plot it all together
plot(x, y_line, label = "line")
scatter!(x_line, y_line_n, label = "line data")
plot!(x, y_sigmoid, label = "sigmoid")
scatter!(x_sigmoid, y_sigmoid_n, label = "sigmoid data")

# Question:

If you take the sigmoid data, which fit is better? The linear or sigmoid?

How uncertain are the parameters?

At what noise level are the two fits equally likely?

# Part 1 (20 points)
Create a function line_fitting(x,y; sigma=[]) that returns the best parameters and the covariance matrix.  The standard deviation of the measurement error, sigma, can be supplied as a number (assuming that all measurements share the same error) or as a vector with the same length as y. If sigma (standard deviation of measurement error) is not specified (sigma=[]), estimate the measurement error using the sum of least squares to estimate $\mathcal{X}^2$.

In [64]:
function line_fitting(x, y; sigma=[])
    #Parameters initial guess
    p0 = [1.0, 2.0]

    #Fit the model to the data
    fit = curve_fit(line_fit, x, y, p0)
    #Best-fit parameters
    p_fit = fit.param

    #Define the residuals function
    local_residuals(p) = y .- line_fit(x, p)

    #Estimation of sigma if it is not given
    if isempty(sigma)
        #Calculate residuals and use their standard deviation as the error estimate
        residuals = local_residuals(p_fit)
       
        #Estimate sigma as the std of residuals (least squares)
        sigma_value = std(residuals)
        sigma = sigma_value * ones(length(x))  # Assign sigma for each data point
       
        println("Estimated sigma: ", sigma[1])
    end
   
    #Calculate chi-squared
    chi2 = p -> sum((local_residuals(p) ./ sigma).^2)

    #Hessian matrix
    hess = ForwardDiff.hessian(chi2, p_fit)
   
    #Covariance matrix (inverse of the Hessian matrix)
    covariance_matrix = inv(2 * hess)
   
    #Errors (standard deviations of the parameters)
    errors = sqrt.(diag(covariance_matrix))
   
    #Final chi-squared value
    chi2_val = chi2(p_fit)

    return p_fit, covariance_matrix, chi2_val, errors
end

line_fitting (generic function with 1 method)

In [67]:
x = 10 .* rand(200)
p = [1.0, 2.0]
line_fit(x, p) = p[1] .+ p[2] .* x
y_exp = line_fit(x, p) .+ 0.1 .* randn(length(x))

#Call the line_fitting function with the data (if sigma is not specified, the estimated sigma will be shown)
p_fit, covariance_matrix, chi2_val, errors = line_fitting(x, y_exp)

println("Parameters: ", p_fit)
println("Covariance Matrix: ", covariance_matrix)
println("Chi2 value: ", chi2_val)
println("Errors: ", errors)
reduced_chi2 = chi2_val ./ (length(y_exp) .- length(p_fit))
println("Reduced chi-squared: ", reduced_chi2)


Estimated sigma: 0.09528172098044049
Parameters: [1.0043035703464787, 1.9975504306648506]
Covariance Matrix: [4.432650300575068e-5 -7.0338568942292124e-6; -7.033856894229213e-6 1.5002357679045474e-6]
Chi2 value: 199.00000000000006
Errors: [0.00665781518260688, 0.0012248411194536815]
Reduced chi-squared: 1.0050505050505054


# Part 2 (20 points)
Similarily, create a function sigmoid_fitting(x,y; sigma=[]) that returns the best parameters and the covariance matrix. Treat sigma the same way as in Part 1.

In [68]:
using LsqFit, Plots, Distributions, Random, Statistics, NonlinearSolve, ForwardDiff, LinearAlgebra
sigmoid_fit(x, p) = p[1] .+ p[2] ./ (1 .+ exp.(-p[3] .* (x .- p[4])))

function sigmoid_fitting(x, y; sigma=[])
    # Initial guess for parameters [offset, scale, steepness, midpoint]
    p0 = [0.0, 1.0, 1.0, 5.0]  # Adjust initial guess as needed

    # Fit the model to the data
    fit = curve_fit(sigmoid_fit, x, y, p0)
    # Best-fit parameters
    p_fit = fit.param

    # Define the residuals function
    local_residuals(p) = y .- sigmoid_fit(x, p)  # Use sigmoid_fit here

    # Estimation of sigma if it is not given
    if isempty(sigma)
        # Calculate residuals and use their standard deviation as the error estimate
        residuals = local_residuals(p_fit)
       
        # Estimate sigma as the std of residuals (least squares)
        sigma_value = std(residuals)
        sigma = sigma_value * ones(length(x))  # Assign sigma for each data point
       
        println("Estimated sigma: ", sigma[1])
    end
   
    # Calculate chi-squared
    chi2 = p -> sum((local_residuals(p) ./ sigma).^2)

    # Compute Hessian matrix using ForwardDiff
    hess = ForwardDiff.hessian(chi2, p_fit)
   
    # Covariance matrix (inverse of the Hessian matrix)
    covariance_matrix = inv(2 * hess)
   
    # Errors (standard deviations of the parameters)
    errors = sqrt.(diag(covariance_matrix))
   
    # Final chi-squared value
    chi2_val = chi2(p_fit)

    return p_fit, covariance_matrix, chi2_val, errors
end

sigmoid_fitting (generic function with 1 method)

In [70]:
x = 10 .* rand(200)
p = [2.0, 1.0, 3.0, 5.0]  # Example parameters for the sigmoid model
y_exp = p[1] .+ p[2] ./ (1 .+ exp.(-p[3] .* (x .- p[4]))) .+ 0.1 .* randn(length(x))

# Call the sigmoid_fitting function with the data (if sigma is not specified, the estimated sigma will be shown)
p_fit, covariance_matrix, chi2_val, errors = sigmoid_fitting(x, y_exp)

println("Parameters: ", p_fit)
println("Covariance Matrix: ", covariance_matrix)
println("Chi2 value: ", chi2_val)
println("Errors: ", errors)
reduced_chi2 = chi2_val / (length(y_exp) - length(p_fit))
println("Reduced chi-squared: ", reduced_chi2)

Estimated sigma: 0.10481378023121948
Parameters: [1.9957183404299408, 1.0163492369215665, 2.6207255072534155, 5.00142693664567]
Covariance Matrix: [3.6593514399374835e-5 -3.915222439708643e-5 0.00026425996548565007 4.523243311969598e-5; -3.915222439708642e-5 8.394874682657246e-5 -0.00048219632346817667 -5.391559076853324e-6; 0.00026425996548565023 -0.0004821963234681767 0.013171658236892751 0.00041859145293560857; 4.523243311969598e-5 -5.391559076853328e-6 0.00041859145293560716 0.00039699180319909157]
Chi2 value: 198.99999999999997
Errors: [0.00604925734279629, 0.00916235487342487, 0.11476784496056704, 0.019924653151287015]
Reduced chi-squared: 1.0153061224489794


# Part 3 (30 points)
Create a function p_ratio_sig_line(x,y; sigma=[]) that returns the Posterior ratio between a fit to a sigmoid and to a line.  Treat sigma the same way as in Part 1-2.

In [71]:
# Define models
line_fit(x, p) = p[1] .* x .+ p[2]
sigmoid_fit(x, p) = p[1] .+ p[2] ./ (1 .+ exp.(-p[3] .* (x .- p[4])))

# Posterior ratio between sigmoid and line fit
function p_ratio_sig_line(x, y; sigma=[])
    function fit_model(fit_func, p0)
        fit = curve_fit(fit_func, x, y, p0)
        p_fit = fit.param
        residuals = y .- fit_func(x, p_fit)
        σ = isempty(sigma) ? std(residuals) * ones(length(x)) :
             (length(sigma) == 1 ? fill(sigma, length(x)) : sigma)
        chi2 = sum((residuals ./ σ).^2)
        logL = -0.5 * chi2
        bic = length(p_fit) * log(length(y)) - 2 * logL
        return bic
    end

    bic_line = fit_model(line_fit, [1.0, 1.0])
    bic_sigmoid = fit_model(sigmoid_fit, [1.0, 2.0, 1.0, 5.0])
    return exp((bic_line - bic_sigmoid) / 2)
end



p_ratio_sig_line (generic function with 1 method)

In [74]:
x = 10 .* rand(200)
p_sig = [1.0, 2.0, 1.5, 5.0]
y = sigmoid_fit(x, p_sig) .+ 0.1 .* randn(length(x))

posterior_ratio = p_ratio_sig_line(x, y)
println("Posterior Ratio (Sigmoid vs Line): ", posterior_ratio)

Posterior Ratio (Sigmoid vs Line): 0.004999999999999731


# Part 4 (30 points)
Create a function sig_p_ratio_is_one(x,y; sigma=[]) that returns the measurement error (either as a number or array depending on how the sigma is supplied) that would result in a posterior ratio between sigmoid and line of one. If sigma is not provided (sigma=[]), proceed as in Part 1-3.

In [ ]:
using LsqFit, ForwardDiff, Statistics, Optim

function sig_p_ratio_is_one(x, y; sigma=[])
    #Define models
    line_fit(x, p) = p[1] .* x .+ p[2]
    sigmoid_fit(x, p) = p[1] .+ p[2] ./ (1 .+ exp.(-p[3] .* (x .- p[4])))

    #BIC calculation
    function compute_bic(fit_func, p0, x, y, sigma)
        fit = curve_fit(fit_func, x, y, p0)
        residuals = y .- fit_func(x, fit.param)
        chi2 = sum((residuals ./ sigma).^2)
        log_likelihood = -0.5 * chi2
        k = length(fit.param)
        bic = k * log(length(y)) - 2 * log_likelihood
        return bic
    end

    #Estimate sigma if not provided
    if isempty(sigma)
        p0_line = [1.0, 1.0]
        fit_line = curve_fit(line_fit, x, y, p0_line)
        residuals = y .- line_fit(x, fit_line.param)
        sigma = std(residuals)
    end

    #Function to find sigma that makes posterior ratio = 1
    function diff_in_bic(log_sigma)
        test_sigma = fill(exp(log_sigma), length(y))
        bic_line = compute_bic(line_fit, [1.0, 1.0], x, y, test_sigma)
        bic_sigmoid = compute_bic(sigmoid_fit, [1.0, 2.0, 1.0, 5.0], x, y, test_sigma)
        return (bic_line - bic_sigmoid) / 2  # log posterior ratio
    end

    #Optimize log(sigma) to find where posterior ratio = 1
    result = Optim.optimize(x -> abs(diff_in_bic(x)), log(1e-3), log(10.0))
    optimal_log_sigma = Optim.minimizer(result)
    optimal_sigma = exp(optimal_log_sigma)

    return fill(optimal_sigma, length(y))
end

sig_p_ratio_is_one (generic function with 1 method)

In [59]:
x = 10 .* rand(200)
true_params = [1.0, 2.0, 1.5, 5.0]
sigmoid_fit(x, p) = p[1] .+ p[2] ./ (1 .+ exp.(-p[3] .* (x .- p[4])))
y = sigmoid_fit(x, true_params) .+ 0.1 .* randn(length(x))

sigma_balance = sig_p_ratio_is_one(x, y)
println("Estimated sigma where posterior ratio is 1: ", sigma_balance[1])


Estimated sigma where posterior ratio is 1: 1.1732533487047951


In [35]:
import Pkg; Pkg.add("Roots")

   Resolving package versions...
   Installed Roots ─ v2.2.7
    Updating `C:\Users\steal\Downloads\Zero2Hero-JuliaWorkshop-main\BME-502-2025-homeworks\Project.toml`
  [f2b01f46] + Roots v2.2.7
    Updating `C:\Users\steal\Downloads\Zero2Hero-JuliaWorkshop-main\BME-502-2025-homeworks\Manifest.toml`
  [f2b01f46] ↑ Roots v2.2.5 ⇒ v2.2.7
Precompiling project...
   7314.4 ms  ✓ Roots
   1928.9 ms  ✓ Roots → RootsChainRulesCoreExt
   3125.1 ms  ✓ Roots → RootsForwardDiffExt
   8747.8 ms  ✓ Bijectors
   4344.0 ms  ✓ Bijectors → BijectorsEnzymeCoreExt
   4536.7 ms  ✓ Bijectors → BijectorsForwardDiffExt
   4719.9 ms  ✓ Bijectors → BijectorsDistributionsADExt
   4762.1 ms  ✓ Bijectors → BijectorsLazyArraysExt
   7129.4 ms  ✓ Bijectors → BijectorsTrackerExt
   6381.8 ms  ✓ AdvancedVI
  20305.4 ms  ✓ DynamicPPL
   5555.0 ms  ✓ DynamicPPL → DynamicPPLChainRulesCoreExt
   5643.9 ms  ✓ DynamicPPL → DynamicPPLEnzymeCoreExt
   5807.9 ms  ✓ DynamicPPL → DynamicPPLForwardDiffExt
   7821.9 ms  ✓ DynamicP

This is a quicker alternative to the above code for Part 4:

In [60]:
using LsqFit, Statistics

function p_ratio_sig_line(x, y; sigma, lambda_beta=1.0)
    #Define models
    line_fit(x, p) = p[1] .* x .+ p[2]
    sigmoid_fit(x, p) = p[1] .+ p[2] ./ (1 .+ exp.(-p[3] .* (x .- p[4])))

    #Helper for BIC
    function compute_bic(fit_func, p0, x, y, sigma)
        fit = curve_fit(fit_func, x, y, p0)
        residuals = y .- fit_func(x, fit.param)
        chi2 = sum((residuals ./ sigma).^2)
        logL = -0.5 * chi2
        k = length(fit.param)
        n = length(y)
        bic = k * log(n) - 2 * logL
        return bic
    end

    #Making sure sigma is an array
    if !isa(sigma, AbstractArray)
        sigma = fill(sigma, length(y))
    end

    #Compute BICs
    bic_line = compute_bic(line_fit, [1.0, 1.0], x, y, sigma)
    bic_sigmoid = compute_bic(sigmoid_fit, [1.0, 2.0, 1.0, 5.0], x, y, sigma)

    #Return posterior ratio
    return exp(-0.5 * (bic_sigmoid - bic_line))
end


p_ratio_sig_line (generic function with 1 method)

In [63]:
using Roots

function sig_p_ratio_is_one(x, y; sigma = [], lambda_beta=1.0)
    f(s) = p_ratio_sig_line(x, y; sigma=s, lambda_beta=lambda_beta) - 1
    root = find_zero(f, 1.0, Order1())  # Or use Bisection() with interval
    return root
end

# Example test
x = 10 .* rand(200)
true_params = [1.0, 2.0, 1.5, 5.0]
sigmoid_fit(x, p) = p[1] .+ p[2] ./ (1 .+ exp.(-p[3] .* (x .- p[4])))
y = sigmoid_fit(x, true_params) .+ 0.2 .* randn(length(x))

sol = sig_p_ratio_is_one(x, y)
println("Sigma where posterior ratio is 1: ", sol)

ratio = p_ratio_sig_line(x, y; sigma=sol)
println("Posterior ratio at that sigma: ", ratio)


Sigma where posterior ratio is 1: 1.2130972659969494
Posterior ratio at that sigma: 1.0


## Programming Advice:
To make your program more efficient you should think about creating functions that provide information that you can use for all of the parts.  Don't write each function individually, but think what are the common task that need to be done, and then call these more general functions, to get you the result for the specific questions.  I will reward good programming style with a bonus 10 points total.